<a href="https://colab.research.google.com/github/Gayatridhote123/FMML_project_and_lab/blob/main/Copy_of_FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?

  The accuracy of a validation set is affected by increasing or reducing the percentage of the validation set in several ways, but it's important to understand that the actual accuracy is not directly dependent on the size of the validation set.
* Increasing the Percentage of the Validation Set:
Stability of Accuracy Estimate: When you increase the size of the validation set, the accuracy estimate becomes more stable and reliable. This is because a larger validation set provides a more representative sample of the overall data distribution, reducing the impact of outliers or anomalies in the evaluation process.
Reduced Training Data: Increasing the validation set size reduces the amount of training data available. This can negatively affect model performance if the model is data-hungry or if the training set becomes too small, leading to underfitting or insufficient learning.
* Decreasing the Percentage of the Validation Set:
Variability in Accuracy Estimate: A smaller validation set may result in a less stable and more variable estimate of accuracy. If the validation set is too small, random variations (or noise) in the data can disproportionately affect the accuracy, potentially leading to inaccurate or misleading results.
More Training Data: On the flip side, decreasing the validation set size increases the size of the training set, which might improve the model's performance by giving it more data to learn from. However, this could come at the cost of overfitting, as the model might perform very well on the training data but poorly on unseen data due to lack of sufficient validation.

2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?

* Training Set Size:
More Training Data:
Improved Model Performance: A larger training set generally leads to a better-performing model because the model has more examples to learn from. This helps the model generalize better to unseen data, improving the accuracy on both the validation and test sets.
With more training data, the model is less likely to overfit to specific patterns in the data. This means that the accuracy on the validation set will be more representative of the accuracy on the test set because the model has learned more generalized patterns.

* Less Training Data:
Potential Overfitting or Underfitting: A smaller training set may lead to overfitting (the model learns specific patterns that do not generalize well) or underfitting (the model doesn't learn enough to generalize). Both issues can cause the validation accuracy to differ significantly from the test accuracy, making the validation set a less reliable predictor of test performance.

 Validation Set Size:

  Larger Validation Set:

* More Reliable Accuracy Estimate: A larger validation set provides a more representative sample of the overall data distribution, making the validation accuracy a more reliable predictor of test accuracy. Since the validation set is more comprehensive, it reduces the likelihood that validation accuracy is overly influenced by noise or outliers.

* Decreased Training Data: However, increasing the validation set size reduces the amount of data available for training. If the training set becomes too small as a result, the model might underperform, and the validation accuracy may not align well with test accuracy because the model hasn't learned enough from the training data.

* Smaller Validation Set:

 Less Reliable Accuracy Estimate: A smaller validation set increases the variability of the accuracy estimate. This can make the validation accuracy less reliable as a predictor of test accuracy because the smaller validation set may not fully represent the diversity in the overall data distribution.
Increased Training Data: On the positive side, more data remains for training, which might improve model performance if the validation set is still sufficiently large to provide meaningful evaluations. However, if the validation set is too small, the test accuracy may deviate from the validation accuracy due to an insufficiently robust validation process.




3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

  A good percentage to reserve for the validation set typically falls between 10% to 30% of the total dataset. The exact percentage depends on the size of your dataset and the complexity of your model.

* For Larger Datasets (e.g., tens of thousands of samples or more):

  10% to 20% of the data for validation is often sufficient.
Why: With a large number of samples, even a smaller percentage provides enough data to accurately assess the model’s performance. The training set will still be large enough for the model to generalize well, and the validation set will be representative of the test set, ensuring a reliable estimate of the model's performance.

* For Smaller Datasets (e.g., less than a few thousand samples):

  20% to 30% of the data for validation may be necessary.
Why: When working with a smaller dataset, a larger validation set is needed to provide a reliable estimate of the model’s performance because each sample represents a more significant portion of the overall data distribution. However, care must be taken not to reduce the training set size too much, as this could cause the model to underfit.

* Balancing Factors:

  Data Size: The larger the dataset, the smaller the percentage of validation data needed, as the validation set will still have enough samples to be representative.

  Model Complexity: More complex models (e.g., deep neural networks) may require more training data to perform well, so the validation set percentage might need to be adjusted accordingly. For example, in deep learning, it's common to reserve 10%-15% for validation because deep models are data-hungry.

  Cross-Validation: If you're unsure of the best split or have a very limited dataset, k-fold cross-validation can help provide more reliable accuracy estimates without reducing the training set. In this case, you may use smaller validation splits within each fold (e.g., 10%-20%).

*  Rule of Thumb:

  10%-20% validation for large datasets.

  20%-30% validation for smaller datasets.


Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

  To implement and compare a 1-nearest neighbor (1-NN) classifier and a 3-nearest neighbor (3-NN) classifier, we will use KNeighborsClassifier and accuracy_score from the scikit-learn library.

    # Import necessary libraries

    from sklearn.datasets import load_iris  # You can replace this with any dataset

    from sklearn.model_selection import train_test_split

    from sklearn.neighbors import KNeighborsClassifier

    from sklearn.metrics import accuracy_score

    # Load a dataset (using the Iris dataset for this example)

    data = load_iris()

    X = data.data  # Features

    y = data.target  # Labels

    # Split the dataset into training and testing sets

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize 1-NN and 3-NN classifiers

    knn_1 = KNeighborsClassifier(n_neighbors=1)

    knn_3 = KNeighborsClassifier(n_neighbors=3)

    # Train the 1-NN classifier

    knn_1.fit(X_train, y_train)

    # Train the 3-NN classifier

    knn_3.fit(X_train, y_train)

    # Make predictions with the 1-NN classifier

    y_pred_1 = knn_1.predict(X_test)

    # Make predictions with the 3-NN classifier

    y_pred_3 = knn_3.predict(X_test)

    # Calculate the accuracy of both classifiers

    accuracy_1 = accuracy_score(y_test, y_pred_1)

    accuracy_3 = accuracy_score(y_test, y_pred_3)

    # Print the results

    print(f"Accuracy of 1-Nearest Neighbor: {accuracy_1:.2f}")

    print(f"Accuracy of 3-Nearest Neighbor: {accuracy_3:.2f}")







## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.


   In K-nearest neighbor (K-NN) classification, the accuracy of the classifier can be influenced by the number of splits in the data (i.e., the training-test split), the split size, and the number of neighbors used (e.g., 1-NN vs. 3-NN). Let’s break down how these factors interact:

1. Number of Splits and Split Size:

  Number of Splits: This refers to how many times the data is split into training and test sets, often done through cross-validation (e.g., k-fold cross-validation). For example, in 5-fold cross-validation, the data is split into 5 portions, and the model is trained and tested 5 times, each time with a different fold being used as the test set and the others as training sets.
Split Size: This is the ratio of training data to test data. For instance, in an 80/20 split, 80% of the data is used for training, and 20% is used for testing.

  Effect on Accuracy:

* Training Size: The larger the training size, the more data the model has to learn from. For K-NN classifiers, which rely heavily on the distribution of training data, larger training sizes tend to lead to more accurate models. As the size of the training set increases, the model’s ability to generalize often improves, though at the expense of possibly overfitting on specific data patterns.

* Testing Size: A smaller test set increases variance in performance estimation (due to less data being used to evaluate the model), leading to less reliable accuracy measures. Conversely, larger test sets provide more reliable performance metrics but require larger training sets to maintain model quality.

  Therefore, increasing the number of splits in cross-validation generally improves the reliability of the performance estimate but requires more computational resources. The split size impacts the accuracy, with smaller test sets typically leading to more fluctuating results and larger training sets improving the generalization of the classifier.

2. Comparing 3-NN to 1-NN:

  1-Nearest Neighbor (1-NN): This classifier uses the single closest neighbor to predict the class of a new data point. It is highly sensitive to noise and outliers because it depends on only one training example. As a result, its accuracy can vary significantly depending on the split size and data distribution.

  3-Nearest Neighbor (3-NN): This classifier uses the three closest neighbors to make predictions, typically by majority vote. It tends to be more robust to noise and outliers than 1-NN because it averages over multiple neighbors, reducing the impact of individual noisy points. However, 3-NN may also smooth over small, localized patterns in the data, which could be picked up by 1-NN.

* Accuracy Trends:

  For Smaller Training Sizes: 1-NN may perform better because it picks up fine details and specific patterns in the limited data. However, this also makes it more susceptible to noise. In contrast, 3-NN can help generalize better, although it may fail to capture smaller patterns.

  For Larger Training Sizes: 3-NN tends to outperform 1-NN because the larger amount of data reduces the risk of overfitting to noise. The use of multiple neighbors helps smooth predictions and generally leads to better accuracy, especially when the data has enough samples for each class to be well represented.

This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?

    averaging the validation accuracy across multiple splits generally gives more consistent and reliable results. This is the essence of cross-validation, which aims to mitigate the variability in validation accuracy that may arise from a single train-test split.

    Reduced Variance: When you use a single split of the data (e.g., one training set and one validation set), the validation accuracy can be influenced by how the data was split. Some splits might be "lucky" and result in higher accuracy, while others might be "unlucky" and result in lower accuracy due to the presence of outliers or non-representative samples in the validation set. Averaging across multiple splits helps smooth out this variability and provides a more stable estimate of model performance.

    Better Representation of Data: Different splits allow the model to be validated on different subsets of the data, improving the likelihood that the accuracy estimate generalizes well to new, unseen data. This ensures that the model isn't evaluated on an unrepresentative portion of the dataset, which can sometimes happen with a single split.

    Robustness to Overfitting/Underfitting: By evaluating the model on multiple splits, the averaging helps reveal if the model's performance is consistent across different parts of the dataset. This provides insights into whether the model is overfitting (performing well on some splits but poorly on others) or underfitting (performing poorly across all splits).

    Cross-Validation Approaches:

    k-Fold Cross-Validation: The most commonly used method involves splitting the data into k equal-sized folds. The model is trained on k-1 folds and validated on the remaining fold. This process is repeated k times, with a different fold used for validation each time, and the validation accuracies are averaged. A typical value for k is 5 or 10.

    Stratified k-Fold Cross-Validation: For classification tasks with imbalanced classes, stratified cross-validation ensures that each fold maintains the same class distribution as the overall dataset. This leads to more reliable accuracy estimates when dealing with imbalanced data.

    Leave-One-Out Cross-Validation (LOOCV): This is an extreme form of cross-validation where k equals the number of samples in the dataset. Each sample is used as a validation set exactly once, and the model is trained on the remaining n-1 samples. This can be computationally expensive but provides a very thorough evaluation.

    from sklearn.model_selection import cross_val_score
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.datasets import load_iris

        # Load dataset
        data = load_iris()

        X = data.data

        y = data.target

        # Initialize the KNN classifier with 3 neighbors
        knn = KNeighborsClassifier(n_neighbors=3)

        # Perform 5-fold cross-validation and average the validation accuracies
        cv_scores = cross_val_score(knn, X, y, cv=5)

        # Print each fold's accuracy and the average

        print(f"Validation Accuracies for each fold: {cv_scores}")

        print(f"Average Validation Accuracy: {cv_scores.mean():.2f}")

        from sklearn.model_selection import cross_val_score
        from sklearn.neighbors import KNeighborsClassifier
        from sklearn.datasets import load_iris


2. Does it give more accurate estimate of test accuracy?


    

* Reduced Bias from Single Split:

  When using a single train-test split, the model's validation accuracy can be biased due to the specific data points in the training and validation sets. If the split happens to be non-representative of the overall dataset (e.g., certain classes are over- or under-represented in the validation set), it can give a misleading estimate of test accuracy.

  Cross-validation averages the accuracy across multiple splits, reducing the influence of any particular data split. This leads to a more representative estimate of how the model will perform on unseen test data.

* Improved Generalization Assessment:

  Cross-validation exposes the model to different subsets of the data in the validation process. By training and validating on different portions of the dataset, the model is tested more rigorously across various data patterns.
This improves the estimate of generalization performance (i.e., how well the model will perform on new, unseen data). In other words, the cross-validation process better simulates real-world performance, resulting in a more accurate approximation of test accuracy.

*  Variance Reduction:

  A single split can lead to high variance in the accuracy estimate, meaning that the validation accuracy might fluctuate depending on the specific split of the data. This makes it harder to predict the model's true test performance.

    Cross-validation reduces this variance by combining results from multiple splits. The average performance across multiple folds provides a smoother, more stable estimate of accuracy, which is more likely to match the model’s true performance on the test set.





3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?

* Higher iterations provide a more accurate and consistent estimate of test accuracy by reducing bias and variance in the estimate.

*  Diminishing returns set in after a certain point, typically around 10 folds, where further increasing the number of iterations yields only marginal improvements in accuracy.

*  Computational cost increases with more iterations, so it’s important to balance the need for accuracy with the available resources.

*  5-fold or 10-fold cross-validation is often sufficient for most tasks, providing a good trade-off between estimate accuracy and computational efficiency.

from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris

    # Load dataset
    data = load_iris()
    X = data.data
    y = data.target

    # Initialize the KNN classifier
    knn = KNeighborsClassifier(n_neighbors=3)

    # Perform 5-fold cross-validation
    cv_scores_5 = cross_val_score(knn, X, y, cv=5)
    print(f"Average Validation Accuracy with 5-Fold: {cv_scores_5.mean():.2f}")

    # Perform 10-fold cross-validation
    cv_scores_10 = cross_val_score(knn, X, y, cv=10)
    print(f"Average Validation Accuracy with 10-Fold: {cv_scores_10.mean():.2f}")

    # Perform 20-fold cross-validation
    cv_scores_20 = cross_val_score(knn, X, y, cv=20)
    print(f"Average Validation Accuracy with 20-Fold: {cv_scores_20.mean():.2f}")



4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

  How Increasing Iterations Helps with Small Datasets:

* More Efficient Use of Data:

  Maximizing Training Data: In traditional train-test splits, a fixed portion of the dataset is reserved for validation, which can reduce the amount of data available for training. When you have a small dataset, this can be a problem because every sample is crucial for training the model effectively.

  Cross-Validation Benefit: Cross-validation allows you to maximize the use of your data. By rotating the validation set across different iterations, all data points are used for both training and validation at some point. For example, in 5-fold cross-validation, 80% of the data is used for training and 20% for validation in each iteration, but each data point is part of the training set in four iterations and the validation set in one iteration.

*   More Reliable Estimates from Small Validation Sets:

  Issue with Small Validation Sets: If you have a small dataset, reserving a fixed portion (like 20%) for validation might not give you enough validation data to provide a reliable estimate of the model’s performance. The small validation set might not be representative of the entire data distribution.

  Cross-Validation Benefit: Increasing the number of iterations means that you validate on different subsets of the data in each iteration. By averaging the performance across multiple validation sets, you reduce the risk of getting a biased estimate from a single, non-representative validation set.

*  The Limitation: Trade-offs in Very Small Datasets

  While increasing iterations helps, there are still some trade-offs and considerations when dealing with very small datasets:

  Potential Overfitting:

  High Variance Models: Small datasets, even when used in cross-validation, can lead to models that overfit because the training sets are still small. This means that the model might perform well on the training data but fail to generalize to unseen data.

  Cross-Validation’s Role: Cross-validation helps detect overfitting by testing the model on multiple validation sets, but it can’t completely eliminate the risk if the dataset is too small. The final averaged accuracy might still be optimistic if the model memorizes the training data due to its limited size.

  Increased Computational Cost:

  Higher Iterations: Increasing the number of folds, while beneficial, also means higher computational costs. With very small datasets, the gains from increasing the number of iterations beyond a certain point (e.g., 10 or 20 folds) might not justify the additional computation, especially if the model is complex.

* Leave-One-Out Cross-Validation (LOOCV):

  LOOCV: For very small datasets, you might consider using leave-one-out cross-validation (LOOCV), where each data point is used as the validation set once and the model is trained on all remaining points. LOOCV provides an exhaustive use of the data, but it's computationally expensive, especially for large datasets, and can lead to higher variance because each validation set contains only one data point

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %
